# Average by genre

## Importation de modules

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import os
import sys
project_dir = os.getcwd().split('notebooks')[0]
sys.path.append(project_dir)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils.tools import *
from src.utils.metrics import *
from src.utils.extract_data import transform_parquet_to_csv
from src.utils.metrics_plot import *
import numpy as np
from numpy.linalg import norm
from sklearn.model_selection import train_test_split
from sklearn import metrics

C:\Users\22sir\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Dataframe des métriques

In [2]:
metrics=['Accuracy','Precision micro', 'Recall micro', 'F1 score micro', 'Hamming-Loss']
final_res=pd.DataFrame(columns=metrics)

## Importation des données et mise en forme des dataframes

In [3]:
#df, le dataframe qu'on utilise avec les features décomposées
df=pd.read_csv('dataset.csv')


#variables utiles
n_titre = len(df) #nombre titres dans la base
n_audio = 256 #nombre de données audio
n_usage = 128 #nombre de données usage
n_label = 21 #nombre labels dans la base
p=1 #proportion de titres de la base utilisés

#dataframe des labels
labels = df.columns[1:22]
df_labels = pd.DataFrame(data=labels,columns=['Label'])

In [4]:
#Ajout colonne nombre de labels pour chaque titre
df=df.drop(columns = ['song_index'])
df["Nombre labels"] = df.iloc[:,0:21].sum(axis=1)

In [5]:
# df_unilabel: la base avec les titres ne comportant qu'un seul label
df= df[df["Nombre labels"] == 1]
df.drop(columns = ['Nombre labels'])

,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127,usage_feature_128
0,0,0,0,0,0,0,0,0,0,0,...,0.008582,-0.000024,-0.272207,0.210503,0.030972,0.080964,-0.015423,0.102148,0.365081,-0.066897
1,0,0,0,0,0,0,0,0,0,1,...,-0.182309,-0.051225,-0.004617,-0.005726,-0.038334,-0.063972,0.053881,-0.001152,0.045032,0.027505
2,0,0,0,0,1,0,0,0,0,0,...,-0.044620,-0.132282,-0.141684,-0.106271,-0.189065,-0.229804,0.083824,-0.160790,0.040417,-0.094503
3,0,0,0,0,0,0,0,0,0,1,...,-0.127573,-0.068515,-0.106374,-0.102694,-0.066937,0.026911,-0.032158,0.063670,-0.026653,-0.068251
4,0,0,0,0,0,0,0,0,0,0,...,0.024463,-0.052952,-0.059056,0.041670,-0.025673,-0.000446,-0.003438,0.060359,0.058938,0.029523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50557,0,0,0,0,1,0,0,0,0,0,...,-0.034780,0.112294,-0.084898,-0.191570,0.153120,0.106828,0.056458,0.127816,0.086341,-0.024928
50558,0,0,0,0,0,0,0,1,0,0,...,0.063418,0.166352,0.021962,0.003740,0.146321,-0.094655,-0.118755,-0.008160,-0.068332,0.041923
50559,0,0,0,0,0,0,0,0,0,0,...,-0.000624,-0.037719,0.018047,0.028091,-0.000380,0.015697,-0.035737,-0.022669,-0.003089,-0.014504
50560,0,0,0,0,0,0,0,0,0,0,...,0.063921,0.000640,-0.081422,-0.012015,-0.046315,-0.112069,0.017265,-0.063093,-0.134029,-0.007111


## Séparation train-test

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

Y = df[labels]
X = df[["usage_feature_"+str(i) for i in range(1,n_usage+1)]+["audio_feature_"+str(i) for i in range(1,n_audio+1)]]
columns = X.columns
index = X.index

In [67]:
#normalisation L2 de X
X = normalize(X, norm='l2')
X = pd.DataFrame(X)
X.columns = columns
X.index=index
X

,usage_feature_1,usage_feature_2,usage_feature_3,usage_feature_4,usage_feature_5,usage_feature_6,usage_feature_7,usage_feature_8,usage_feature_9,usage_feature_10,...,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127,usage_feature_128
106,-0.003502,0.009473,-0.018925,0.011646,-0.007872,-0.067682,0.014128,0.011741,-0.065874,0.024521,...,-0.043221,-0.085039,0.081496,-0.026725,-0.045337,0.023345,0.024011,-0.058067,-0.115079,-0.057350
10368,0.010136,0.003542,-0.001883,-0.013543,-0.017521,0.042429,0.004291,-0.004314,-0.045055,-0.046551,...,0.085310,-0.012038,-0.083601,-0.029201,-0.006609,-0.013994,-0.025269,0.115839,0.093747,0.032425
43365,-0.000197,-0.001665,0.012022,0.000917,-0.010417,0.002932,-0.004368,-0.028730,-0.004892,-0.009975,...,-0.076844,-0.004982,0.097929,-0.037737,-0.061498,-0.008912,-0.026940,-0.016152,-0.018855,-0.069888
48952,-0.001687,0.005166,0.029799,-0.030831,0.040375,-0.027395,-0.041321,0.024693,0.005310,-0.004002,...,-0.057874,0.001151,0.034447,-0.049363,0.255599,-0.098947,0.027389,-0.047128,-0.102305,0.065255
44195,0.004750,-0.000356,-0.014034,-0.000366,0.011909,0.016726,0.016942,-0.011188,0.000881,-0.018268,...,0.054024,0.036367,0.061509,0.030993,0.089347,-0.085786,0.071931,-0.049408,-0.052711,0.025121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40126,0.001796,0.004779,-0.013640,0.002435,-0.008807,-0.006644,0.006608,0.001958,-0.008259,-0.008003,...,0.215605,-0.006166,0.161655,-0.015267,0.016311,0.074686,-0.014156,0.003282,-0.084867,-0.059461
925,-0.001001,0.000387,-0.001753,0.015873,0.000626,-0.003287,0.002501,-0.014719,-0.007404,0.008646,...,0.051511,-0.016645,0.042270,-0.013707,0.000044,-0.023756,-0.026676,0.011631,-0.025327,-0.001093
40465,0.006706,0.000159,-0.030664,-0.002571,0.014141,0.020005,0.014615,-0.007336,0.002238,-0.009064,...,0.056915,-0.019216,-0.009356,-0.049683,0.050086,-0.037777,-0.030078,-0.010390,-0.022488,0.118210
7178,0.004432,0.018859,-0.012895,-0.025325,-0.015036,-0.003340,0.020675,-0.003962,-0.022983,-0.005005,...,0.046291,0.201534,0.122094,-0.042544,0.018647,0.027143,-0.244152,0.134975,0.055604,-0.072483


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=18)

In [9]:
#le dataframe d'entraînement avec les sorties Y et les features X associées à chaque titre
df_train = pd.concat([X_train, y_train], axis = 1)

## Calcul des features moyennes pour chaque label

In [13]:
#le data frame avec les features moyennes de chaque label (moyenne calculée sur l'échantillon d'entraînement)
df_moyenne = pd.DataFrame(columns=df_train.columns)
for label in labels:
    df_moyenne.loc['Average_' + label] = df_train[df_train[label] == 1].mean(axis = 0)

In [15]:
#On ne garde que les moyennes des features
df_moyenne = df_moyenne[["usage_feature_"+str(i) for i in range(1,n_usage+1)]+["audio_feature_"+str(i) for i in range(1,n_audio+1)]]
df_moyenne

,usage_feature_1,usage_feature_2,usage_feature_3,usage_feature_4,usage_feature_5,usage_feature_6,usage_feature_7,usage_feature_8,usage_feature_9,usage_feature_10,...,audio_feature_247,audio_feature_248,audio_feature_249,audio_feature_250,audio_feature_251,audio_feature_252,audio_feature_253,audio_feature_254,audio_feature_255,audio_feature_256
Average_asian,-0.001813,0.002964,0.007558,-0.013170,0.001070,-0.000551,0.013556,-0.002588,-0.015186,0.026233,...,0.070433,0.014264,0.012155,-0.103778,-0.494777,0.303398,-0.228496,0.157736,0.758998,-0.434382
Average_rnb,-0.000429,0.002114,0.000713,0.002589,-0.004937,-0.000998,0.004125,0.000734,-0.006184,-0.014059,...,0.091728,0.093325,-0.193266,0.082557,-0.266792,0.929134,-0.249044,0.188064,0.359308,-0.358009
Average_reggae,-0.000125,0.003694,0.001070,-0.001549,-0.002511,-0.000170,-0.007865,-0.003798,-0.007681,-0.011680,...,0.177879,0.012891,-0.186714,0.121273,-0.295919,0.748319,-0.324451,0.085861,0.280435,-0.128048
Average_blues,-0.006880,0.013536,-0.007689,0.023094,0.018141,-0.005343,0.035752,0.006700,-0.014854,0.009979,...,-0.028488,0.419679,0.055917,0.031609,-0.428509,0.001481,-0.042122,-0.091944,0.298054,-0.258059
Average_pop,-0.002040,0.001065,0.003561,0.004177,0.003471,-0.015551,-0.003179,-0.034894,-0.009776,-0.013419,...,0.144045,0.060368,-0.133008,-0.029596,-0.427657,0.737376,-0.268046,0.267719,0.619775,-0.327144
Average_dance,-0.000954,-0.002970,-0.001261,0.003943,0.003032,-0.003205,-0.006927,-0.034226,-0.005592,0.006795,...,0.579693,0.260924,-0.204807,-0.084701,-0.508237,1.222674,-0.292534,0.262222,0.984668,-0.373698
Average_folk,0.009136,-0.003492,-0.028744,-0.003367,0.014317,0.024601,0.005704,0.008302,0.005101,0.000165,...,0.074295,0.134905,-0.007096,-0.118201,-0.583559,0.092936,0.030518,-0.074344,0.647520,-0.491964
Average_arabic-music,0.002359,0.001243,-0.001010,-0.002260,-0.000674,0.006363,0.000769,-0.007816,0.003180,-0.003469,...,0.205268,0.023815,-0.174327,0.046922,-0.298320,0.756823,-0.339676,0.198757,0.325981,-0.112863
Average_indie,0.003918,0.003954,-0.012375,-0.002791,0.004244,0.002737,0.001995,-0.009267,-0.008083,0.003975,...,0.161380,0.100189,-0.056784,-0.126581,-0.521359,0.470913,-0.157405,0.009906,0.954994,-0.433596
Average_rock,0.003128,0.002127,-0.010163,-0.000951,0.010205,0.003641,-0.007535,-0.012942,0.002950,-0.001585,...,0.038047,0.055974,0.182052,-0.144109,-0.491067,0.096838,-0.216617,0.015642,0.941283,-0.393911


In [122]:
#le dataframe de test avec les sorties Y et les features X associées à chaque titre
df_test = pd.concat([X_test, y_test], axis = 1)
df_test.head(5)

,usage_feature_1,usage_feature_2,usage_feature_3,usage_feature_4,usage_feature_5,usage_feature_6,usage_feature_7,usage_feature_8,usage_feature_9,usage_feature_10,...,latin,classical,k-pop,brazilian,metal,rap,jazz,electronic,african,country
21054,0.011967,-0.011819,-0.049459,-0.002455,0.024134,0.017773,0.009970,0.025119,0.004892,0.016159,...,1,0,0,0,0,0,0,0,0,0
47782,-0.013764,-0.004428,0.018278,-0.006117,0.033397,-0.006854,-0.019062,-0.047000,0.038519,0.037225,...,0,0,0,0,0,0,0,0,0,0
12447,-0.001614,0.002465,0.000476,-0.000686,-0.004188,0.001361,0.013555,-0.002152,-0.008344,-0.015635,...,0,0,0,0,0,0,0,0,0,1
19272,0.002365,-0.002760,0.001047,0.007628,-0.008013,-0.005495,0.009805,-0.040013,0.008111,0.010035,...,0,0,0,0,0,0,0,0,0,0
28517,-0.001901,0.003157,0.013484,-0.013545,0.002213,-0.015621,0.011370,0.019195,0.021083,0.013702,...,0,0,0,0,0,0,0,0,0,0


In [123]:
df_test1 = df_test[["usage_feature_"+str(i) for i in range(1,n_usage+1)]+["audio_feature_"+str(i) for i in range(1,n_audio+1)]]
df_test1.shape[0]

12801

## Cosine similarity

In [16]:
def cosine1(a,b):
    A = np.array(a)
    B = np.array(b)
 
    return np.dot(A,B)/(norm(A)*norm(B))

def cosine(a,b):
    A = np.array(a)
    B = np.array(b)
 
    return norm(A-B)

## Labels prédits

In [20]:
y=[]
for i in range (X_test.shape[0]):
    s=[]
    for j in range(21):
        a= cosine(df_moyenne.iloc[j,:],X_test.iloc[i,:])
        s.append(a)
    for k in range(len(s)):
        if s[k]== min(s):
            y.append(k)

In [18]:
df_hat=pd.DataFrame(columns = labels) #le data frame des valeurs prédites
for i in range(len(y)):
    pred = [0]*21
    pred[y[i]]=1
    df_hat.loc[i]=pred
df_hat = df_hat.astype(str).astype(int)

df_hat.index = y_test.index

In [82]:
#cosine normalisé
final_res.loc['Average by genre']=all_metrics(y_train.values,df_hat.values)
final_res

,Accuracy,Precision micro,Recall micro,F1 score micro,Hamming-Loss
Average by genre,0.001,0.001,0.001,0.001,0.095


In [19]:
#norme euclidienne sur les tests, avec usage et audio
final_res.loc['Average by genre']=all_metrics(y_test.values,df_hat.values)
final_res

,Accuracy,Precision micro,Recall micro,F1 score micro,Hamming-Loss
Average by genre,0.487,0.487,0.487,0.487,0.049
